# Web Scraping Deep Dive


Lesson Goals

    Learn how to handle different response status codes.
    Learn how to handle request errors.
    Learn how to define the user agent string in requests.
    Learn how to make asynchronous requests.
    Learn how to limit asynchronous requests rate.
    Understanding robots.txt

Introduction

Now that you have had the basic knowledge about and hands-on experience with web scraping, it's the time to dive deep in. WWW is the biggest man-made data repository in the history. But the format, quality, and stability of the data sources (i.e. websites) are extremely unpredictable. Therefore, you have to learn how to deal with those issues in this imperfect world. In addition, there are many human-imposed rules and conventions we need to follow and respect because we are data analysts not Internet spies.

In this lesson, you will learn about the common challenges and pitfalls in web scraping as well as what techniques you can use in order to get the job done. After mastering those techniques, you will be prepared to scrape the modern web in reality.
Handling Web Response Status Codes

In the API Deep Dive lesson, you have learned about the API response status codes. In web scraping you will encounter the same status codes because those status codes are designed for the whole Internet. The request Python library you have learned previously allows you to detect the web response status code so that you can handle different types of responses accordingly.

Try the command below in your Python console:

In [1]:
import requests
r1 = requests.get('https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/the-html5-breakfast-site.html')
print(r1.status_code)
r2 = requests.get('https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/forbidden')
print(r2.status_code)
r3 = requests.get('http://google.com')
print(r3.history)

print(r3.history[0].status_code)

print(r3.history[0].text)

200
403
[<Response [301]>]
301
<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">
<TITLE>301 Moved</TITLE></HEAD><BODY>
<H1>301 Moved</H1>
The document has moved
<A HREF="http://www.google.com/">here</A>.
</BODY></HTML>



As you see, the requests library allows you to check whether your web request is successful (i.e. with 20x status codes) or not (i.e. with 40x, or 50x status codes). It's a good idea to check the status code before you parse the response text. Once you receive the response from the requests.get method, do something like below: 

In [2]:
import requests

url='http://google.com'

r = requests.get(url) 
if r.status_code < 300:
    print('request was successful')
elif r.status_code >= 400 and r.status_code < 500:
    print('request failed because the resource either does not exist or is forbidden')
else:
    print('request failed because the response server encountered an error')

request was successful


It should be noted that the requests lib automatically makes additional requests to the redirected URL if the web resource is moved (i.e. 30x status codes). Even if a moved web resource redirects once again, the requests lib will track it all the way down until it receives success or failure as long as the number of redirects does not exceed the redirect limit (default 30). You can choose to disallow redirects by using requests.get(url, allow_redirects=False) so that requests will not track down the redirected URL. Or you can choose to reduce the max redirects allowed by using requests.get(url, max_redirects=3) so as to avoid endless redirects or save time in making requests.



# Handling Request Errors

Sometimes requests cannot complete its transaction and you will need to catch the error then handle them accordingly. Some of the common errors you may encounter include:
Exceeing Max Redirects

As previously mentioned, there is a max redirects number by default (30) which you can override with max_redirects. If the number of redirects exceeds the limit, requests will throw a TooManyRedirects exception.


Timeout

Sometimes a remote server is not responsive either because requests cannot connect to the intended web resource or because the remote server does not send back the promised data. If that happens, requests will typically wait for a long period of time until the connection is closed by the remote server then throw a ConnectTimeout exception. This is a big waste of time because most modern websites respond to web requests within a couple of seconds. Therefore, it's a desirable approach to supply a timeout argument to requests to limit the amount of wait time.


SSL Certificate Error

Secure Sockets Layer (SSL) is a standard security protocol for establishing encrypted links between a web server and a browser in an online communication. To know if the browser connection to a website is encrypted or not, simply check if the URL starts with HTTP or HTTPS. The successor of SSL is Transport Layer Security (TLS) which is strongly promoted by tech giants such as Google. For example, Chrome has been marking HTTP webpage as insecure and discouraging users from connecting to those webpages. The efforts from these tech giants are effectively pushing the whole Internet towards adopting SSL/TLS because if a website doesn't adopt the standard it will lose users.

However, if a website wants to use SSL/TLS, it has to purchase a special certificate from certificate vendors and configure the web server properly in order for the certificate to be functional. If the SSL/TLS certificate is not installed properly or it has expired (purchased certificate has to be renewed every two years), modern browsers such as Chrome will indicate the problem to the users. The requests lib, similarly, will throw an exception if it detects the SSL certificate is problematic. 

In [3]:
try:
    r = requests.get(url, timeout=10)
except requests.exceptions.Timeout:
    print ('a')
    # timeout error... do something
#except requests.exceptions.TooManyRedirects:
    # redirect error... do something
#except requests.exceptions.SSLError:
    # ssl error... do something
#except requests.exceptions.RequestException as e:
    # other unknown errors... do something

# User Agent

When a browser makes HTTP/HTTPS requests, there is a special field in the request header called User-Agent that identifies the browser and operating system from which the request is made. Some websites send out different responses to the requests made with different user agents for reasons such as:

    They want to avoid bugs of the website that only occur in certain browsers or browser versions.

    They want to personalize user experience (e.g. desktop vs mobile, small vs big screen, etc.) by sending different data.

    They don't want to send the same responses to search engines as to human users.

Not all websites differentiate user agents and if they do there is often a good reason. You don't need to worry too much about what user agents to use when you request web data. But it's helpful to have this knowledge. In case you decide to fool the website by pretending to be a certain browser, use the approach below. 

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
response = requests.get(url, headers=headers)
print(response.content[0:600])

b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="es"><head><meta charset="UTF-8"><meta content="origin" name="referrer"><meta content="Google.es permite acceder a la informaci\xc3\xb3n mundial en castellano, catal\xc3\xa1n, gallego, euskara e ingl\xc3\xa9s." name="description"><meta content="noodp" name="robots"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><meta content="origin" name="referrer"><title>Google</title><script nonce="+ZDEYVRC7WIBi72wYPh1xA==">(function(){window.google={kEI:\'Qm4jXZ2lDNeLjLsPlMkP\',kEXPI:\'31\',authuser:0,kscs:'


With the uer agent string above, the request pretends to be from Chrome browser v71.0.3578.98 in macOS 10.14.2 (Mojave). To see what the user agent string look like in other browsers/OS, check out this documentation.
Making Asynchronous Requests

When you make massive requests to websites, it can be extremely time-consuming. To complete your request faster, you can take advantage of the async module of requests. Here's how ( you'll need to first install asyncio with pip before you can try the following code ): 

In [5]:
import asyncio, requests

urls = [
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/breakfast.jpg',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/forbidden',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/the-html5-breakfast-site.html'
]

async def main():
    loop = asyncio.get_event_loop()
    futures = [loop.run_in_executor(None, requests.get, url) for url in urls]
    for response in await asyncio.gather(*futures):
        print(response.status_code)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())


RuntimeError: This event loop is already running

# Dealing with Throttling and Rate Limiting

In modern websites especially those having massive users, throttling and/or rate limiting is often enforced so that a certain person cannot make too frequent API/web requests to the website. These approaches are not targeting at regular human users but rather search engine bots and especially hackers. With throttling, the same requester (usually judged by IP address or account) cannot make more requests than the limit allowed within a certain period of time (e.g. 10,000 requests/day). If the limit is exceeded, the requester will receive an error or simply no response. With rate limiting, a requester must control the frequency of the requests under a certain threshold (e.g. 10 requests/second). When you test your web scraping scripts, if you receive a lot of errors in your responses, it does not necessarily mean the web resources are invlaid. It may because the websites you make requests to are throttling or limiting your requests.

The throttling thresholds and wait peroids differ from website to website. In order to know whether you may be throttled by exceeding the limit, you need to check out the user agreement of the website that you make requests to.

To control the request rate of your scripts, you can wait a period of time after making each request by calling time.sleep(). For example:



In [6]:
import requests, time

urls = [
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/breakfast.jpg',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/forbidden',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/the-html5-breakfast-site.html'
]

for url in urls:
    response = requests.get(url)
    print(response.status_code)
    time.sleep(1)

200
403
200
